# Implementation of 9-layer CNN (pytorch)

In [41]:
import os
import torch
import mlflow
import time
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

## Data loaders

In [42]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(128),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(129),
                                      transforms.CenterCrop(128),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                    ])

# setting up data loaders
data_dir = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_224')

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'validation'), transform=test_transforms)

## Defining  9-layer CNN (geetharamani et.al., 2019):

In [43]:
class NineLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.poolconv1 = nn.Conv2d(32, 16, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 3)
        self.poolconv2 = nn.Conv2d(16, 8, 1)
        self.conv3 = nn.Conv2d(8, 8, 3)
        self.fc1 = nn.Linear(8 * 28 * 28, 1568)
        self.fc2 = nn.Linear(1568, 128)
        self.fc3 = nn.Linear(128, 39)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.poolconv1(self.pool(x))
        x = F.relu(self.conv2(x))
        x = self.poolconv2(self.pool(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

model = NineLayerNet()
model

NineLayerNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (poolconv1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (poolconv2): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=6272, out_features=1568, bias=True)
  (fc2): Linear(in_features=1568, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=39, bias=True)
)

## Defining Training function 

In [44]:
def train(model, train_loader, validation_loader, config, n_epochs=10, stopping_treshold=None):

    if torch.cuda.is_available():
        print('CUDA is available!  Training on GPU ...')
        model.cuda()


    # Loss and optimizer setup 
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

    # Setting minimum validation loss to inf
    validation_loss_minimum = np.Inf 
    train_loss_history = []
    validation_loss_history = []

    for epoch in range(1, n_epochs +1):

        t0 = time.time()
        training_loss = 0.0
        validation_loss = 0.0

        # Training loop
        training_accuracies = []
        for X, y in train_loader:
            
            # Moving data to gpu if using 
            if torch.cuda.is_available():
                X, y = X.cuda(), y.cuda()
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # calculate the batch loss
            loss = criterion(output, y)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            training_loss += loss.item()*X.size(0)

            # calculating accuracy
            ps = torch.exp(output)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == y.view(*top_class.shape)
            training_accuracies.append(torch.mean(equals.type(torch.FloatTensor)).item())

        # Validation Loop
        with torch.no_grad():
            accuracies = []
            for X, y in validation_loader:

                # Moving data to gpu if using 
                if torch.cuda.is_available():
                    X, y = X.cuda(), y.cuda()
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model(X)
                # calculate the batch loss
                loss = criterion(output, y)
                # update validation loss
                validation_loss += loss.item()*X.size(0)

                # calculating accuracy
                ps = torch.exp(output)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == y.view(*top_class.shape)
                accuracies.append(torch.mean(equals.type(torch.FloatTensor)).item())
                
        # Mean loss 
        mean_training_loss = training_loss/len(train_loader.sampler)
        mean_validation_loss = validation_loss/len(validation_loader.sampler)
        mean_train_accuracy = sum(training_accuracies)/len(training_accuracies)
        mean_accuracy = sum(accuracies)/len(accuracies)
        train_loss_history.append(mean_training_loss)
        validation_loss_history.append(mean_validation_loss)
        t_d0 = time.time() - t0

        # Printing epoch stats
        print(f'Epoch: {epoch}/{n_epochs}, ' +\
              f'Training Loss: {mean_training_loss:.3f}, '+\
              f'Train accuracy {mean_train_accuracy:.3f} ' +\
              f'Validation Loss: {mean_validation_loss:.3f}, '+\
              f'Validation accuracy {mean_accuracy:.3f},' +\
              f'Epoch runtime: {t_d0}')  

        # logging with mlflow 
        if mlflow.active_run():
            mlflow.log_metric('loss', mean_training_loss, step=epoch)
            mlflow.log_metric('accuracy', mean_train_accuracy, step=epoch)
            mlflow.log_metric('validation_accuracy', mean_accuracy, step=epoch)
            mlflow.log_metric('validation_loss', mean_validation_loss, step=epoch)

        # Testing for early stopping
        if stopping_treshold:
            if mean_validation_loss < validation_loss_minimum:
                validation_loss_minimum = mean_validation_loss
                print('New minimum validation loss (saving model)')
                save_pth = os.path.join('models',f'{config["name"]}.pt')
                torch.save(model.state_dict(), save_pth)
            elif len([v for v in validation_loss_history[-stopping_treshold:] if v > validation_loss_minimum]) >= stopping_treshold:
                print(f"Stopping early at epoch: {epoch}/{n_epochs}")
                break
        

    return train_loss_history, validation_loss_history

### Train config

In [45]:
# Configs 
config = {
    'max_epochs': 200,
    'learning_rate': 0.003,
    'resolution': 128,
    'name': 'CNN_9layer_pytorch'
}

## Training 

In [46]:

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

mlflow.set_experiment("Plant Leaf Disease")

with mlflow.start_run():
    mlflow.log_param('framework', 'pytorch')
    mlflow.log_param('data_split', '90/10')
    mlflow.log_param('type', '9_layer_CNN')
    mlflow.log_params(config)
    train(model, train_loader, validation_loader, config, n_epochs=config['max_epochs'], stopping_treshold=20)

CUDA is available!  Training on GPU ...
Epoch: 1/200, Training Loss: 2.630, Train accuracy 0.285 Validation Loss: 1.721, Validation accuracy 0.483,Epoch runtime: 122.43713927268982
New minimum validation loss (saving model)
Epoch: 2/200, Training Loss: 1.738, Train accuracy 0.490 Validation Loss: 1.263, Validation accuracy 0.601,Epoch runtime: 169.2669279575348
New minimum validation loss (saving model)
Epoch: 3/200, Training Loss: 1.409, Train accuracy 0.578 Validation Loss: 1.063, Validation accuracy 0.665,Epoch runtime: 173.15866374969482
New minimum validation loss (saving model)
Epoch: 4/200, Training Loss: 1.261, Train accuracy 0.621 Validation Loss: 0.823, Validation accuracy 0.731,Epoch runtime: 152.771555185318
New minimum validation loss (saving model)
Epoch: 5/200, Training Loss: 1.162, Train accuracy 0.650 Validation Loss: 0.734, Validation accuracy 0.772,Epoch runtime: 162.3954164981842
New minimum validation loss (saving model)
Epoch: 6/200, Training Loss: 1.076, Train ac